# Predicting the Sale Price of Bullzoders using Machine Learning

in this notebook, we are going to go through an example of a machine 
learning project with the goal of predicting the sale price of 
bulldozers.

## 1. Problem definition
> How well can we predict the future sales price of a bulldozer, given its characteristics and previous examples of how much similar bulldozers have been sold for?

## 2. Data
The data is downloaded from the Kaggle Bluebook for Bulldozers competition: https://www.kaggle.com/c/bluebook-for-bulldozers/data

There are 3 main datasets: 

* Train.csv is the training set, which contains data through the end of 2011.
* 
Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of   the competition. Your score on this set is used to create the public leaderboard.* 
Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. You       r score on the test set determines your final rank for the competitio
  n.
## 3. Evaluatno
The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

For more on the evaluation of this project check: www.kaggle.com/competitions/bluebook-for-bulldozers/overview/evaluation 

**Note:** the goal for most regression evaluation metrics is to minimize the error. For example our goal for this project will be to build a machine learning model which minimizes RMSLE.
n

## 4. Featur

Kaggle provides a data dictionary detailing all of the features of the dataset. You can view this data on excel: https://www.kaggle.com/c/bluebook-for-bulldozers/data?select=Data+Dictionary.xlsx
es

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
# Import training and validation sets
df = pd.read_csv("data/bluebook-for-bulldozers/TrainAndValid.csv", low_memory=False)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000])

In [ ]:
df.saledate[:1000]

In [ ]:
df.SalePrice.plot.hist();

### Parsing dates

When we work with time series data, we would like to enrich the time & date component as much as possible.

We can do that by telling pandas which of our columns has dates in it using the `parse_dates` parameter.


In [ ]:
# Import data again but this time parse dates
df = pd.read_csv("data/bluebook-for-bulldozers/TrainAndValid.csv", low_memory=False, parse_dates=["saledate"])

In [ ]:
df.saledate.dtype

In [ ]:
df["saledate"][:1000]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000]);

In [ ]:
df.head()

In [ ]:
# Transpose to view all the columns individually
df.head().T

In [ ]:
df.saledate.head(20)

### Sort DataFrame by saledate 

When working with time series data, it is a good idea to sort it by date.

In [ ]:
# Sort DataFrame in date order
df.sort_values(by=["saledate"], inplace=True, ascending=True)

### Make a copy of original DataFrame

We make a copy of the original dataframe, so upon manipulation of the dataset we still have original unadultrated data.

In [ ]:
# Make a copy
df_tmp = df.copy()

In [ ]:
df_tmp["saledate"][:20]

### Add datetime parameters for `saledate` column

In [ ]:
df_tmp[:1].saledate.dt.year

In [ ]:
df_tmp[:1].saledate.dt.day

In [ ]:
df_tmp[:1].saledate

In [ ]:
df_tmp["saleYear"] = df_tmp.saledate.dt.year
df_tmp["saleMonth"] = df_tmp.saledate.dt.month
df_tmp["saleDay"]= df_tmp.saledate.dt.day
df_tmp["saleDayOfWeek"]= df_tmp.saledate.dt.dayofweek
df_tmp["saleDayOfYear"]= df_tmp.saledate.dt.dayofyear

In [ ]:
df_tmp[:2].T

In [ ]:
# Now we have enriched our Dataframe with data time features, we can drop the redundant `saledate` column
df_tmp.drop("saledate", axis=1, inplace= True)

In [ ]:
df_tmp.head().T

In [ ]:
# Check the values of different columns
df_tmp.state.value_counts()

## 5. Modelling

We have done some EDA,but we can do more, however we can try doing some model driven EDA

In [ ]:
# First let us check the DataFrame, we are trying to determine SalePrice, 
# using other columns as features, so let us allocate SalePrice as X column first.
# Using the scikit learn machine learning map on https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
# We can see we want to predict a quatity so we can classify this problem as a regression problem
df_tmp.head()

In [ ]:
# Lets build a machine learning model. see https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn-ensemble-randomforestregressor
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, # n_jobs=-1 using as many cores on computer to speed up processing on 412698 rows of data
                              random_state=42)
model.fit(df_tmp.drop("SalesPrice",axis=1),df_tmp["SalesPrice"])

In [ ]:
# we cant start modelling if any of our column datatypes are not numerical datatypes
df.info()

In [ ]:
# For example
df_tmp["UsageBand"].dtype

In [ ]:
# also we cant perform modeling if there are missing data in the columns
df_tmp.isna().sum()

### Convert the String datatypes into categories

One way to turn all our data into numbers is by converting them into pandas categories
We can check out the different datatypes compatible with pandas here
https://pandas.pydata.org/docs/reference/api/pandas.api.types.pandas_dtype.html

In [ ]:
df_tmp.head().T

In [ ]:
pd.api.types.is_object_dtype(df_tmp["UsageBand"])

In [ ]:
# Find the columns which contain strings
for label,content in df_tmp.items():
    if pd.api.types.is_object_dtype(content):
        print(label)
        

In [ ]:
# this will turn all the string values into categories
for label,content in df_tmp.items():
    if pd.api.types.is_object_dtype(content):
        df_tmp[label]= content.astype("category").cat.as_ordered()

In [ ]:
df.info()

In [ ]:
# This will turn all of the string values into category values
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label]=content.astype("category").cat.as_ordered()

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.state.cat.categories

In [ ]:
df_tmp.state.value_counts()

In [ ]:
df_tmp.state.cat.codes

Thanks to pandas categories we now have a way to access all of our data in form of numbers.

But we still need to fix the issue of missing data from the dataset

In [ ]:
# Check missing data
df_tmp.isnull().sum()

In [ ]:
# View as a % of missing data
df_tmp.isnull().sum()/len(df_tmp)

## Save preprocessed data

In [ ]:
# Export current tmp dataframe
df_tmp.to_csv("data/bluebook-for-bulldozers/train_tmp.csv", index=False)

In [ ]:
# Import preprocessed data
df_tmp = pd.read_csv("data/bluebook-for-bulldozers/train_tmp.csv", low_memory=False)
df_tmp.head()

In [ ]:
df_tmp.isna().sum()

## Fill missing values

### Fill numeric missing values first

In [ ]:
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
df_tmp.ModelID

In [ ]:
# Check for which numeric columns have null values
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
# Fill numeric rows with the median
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # Add a binary column which tells us if the data was missing or not
            df_tmp[label+"_is_missing"] = pd.isnull(content)
            # Fill missing numeric values with median
            df_tmp[label]=content.fillna(content.median())

In [ ]:
# Demonstrate how median is more robust than mean
# https://datascience.stackexchange.com/questions/46744/when-to-use-mean-vs-median#:~:text=The%20median%20is%20especially%20useful,are%20usually%20discussed%20using%20medians.
# https://surveymethods.com/when-is-it-generally-better-to-use-median-over-mean/

hundreds = np.full((1000,),100)
hundreds_billion = np.append(hundreds,1000000000)
np.mean(hundreds),np.mean(hundreds_billion),np.median(hundreds),np.median(hundreds_billion)

In [ ]:
# Check if there's any null numeric values
for label,contenn in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
df_tmp.auctioneerID_is_missing.value_counts()

In [ ]:
df_tmp.head().T

In [ ]:
# Check for columns which are not numeric
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
    

In [ ]:
# Turn categorical variables into numbers and fill missing
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        #if pd.isnull(content).sum():
            # Add a binary column to indicate whether sample had missing value
            df_tmp[label+"_is_missing"] = pd.isnull(content)
            # Turn categories into numbers and add +1
            df_tmp[label]= pd.Categorical(content).codes + 1          
        
    

In [ ]:
df_tmp.head().T

In [ ]:
df_tmp.isna().sum()

Now that all of our data is numeric as well as our dataframe has no missing values, we should be able to build a machine learning model